<a href="https://colab.research.google.com/github/JuliKM/Llm-hackathon/blob/main/LlmForConsulateChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install necessary libraries
!pip install langchain
!pip install chromadb
!pip install tiktoken
!pip install unstructured
!pip install openai

In [ ]:
#Get api_key from env
import os
api_key = os.environ.get('API_URL')

In [ ]:
from langchain.document_loaders import UnstructuredURLLoader

urls = [
    "https://consmendoza.esteri.it/consolato_mendoza/it/la_comunicazione/dal_consolato/2023/06/prenot-mi-appuntamenti-anagrafe_14.html",
    "https://consmendoza.esteri.it/consolato_mendoza/it/i_servizi/per_i_cittadini/cittadinanza/ciudania-matrimonio-req.html",
    "https://consmendoza.esteri.it/consolato_mendoza/resource/doc/2021/06/guida_citt_x_mat.pdf",
    "https://consmendoza.esteri.it/consolato_mendoza/es/i_servizi/per_i_cittadini/cittadinanza/ciudadania-matrimonio-doc.html",
    "https://consmendoza.esteri.it/consolato_mendoza/it/",
    "https://docs.google.com/document/d/e/2PACX-1vT8qHba7oGpVWg8FJvToQMgJBGpEib3xyKLHvei_7S2i3Gi5PyFtU6SS7z47AgJQEaNqS0EubNpVYkx/pub",
    "https://consmendoza.esteri.it/consolato_mendoza/it/la_comunicazione/dal_consolato/2023/06/prenot-mi-appuntamenti-anagrafe_14.html",
    "https://consmendoza.esteri.it/consolato_mendoza/it/la_comunicazione/dal_consolato/",
    "https://consmendoza.esteri.it/consolato_mendoza/es",
    "https://consmendoza.esteri.it/consolato_mendoza/resource/img/2023/05/tabella_tariffe_consolari_maggio_2023.pdf",
    ]

loader = UnstructuredURLLoader(urls=urls)

In [ ]:
data = loader.load()

In [ ]:
# Import RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create the text splitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# Split the documents
documents = splitter.split_documents(data)

In [ ]:
# Import tiktoken
import tiktoken

# Create an encoder
encoder = tiktoken.encoding_for_model("text-embedding-ada-002")

# Count tokens in each document
doc_tokens = [len(encoder.encode(doc.page_content)) for doc in documents]

# Calculate the sum of all token counts
total_tokens = sum(doc_tokens)

# Calculate a cost estimate
cost = (total_tokens/1000) * 0.0004
print(f"Total tokens: {total_tokens} - cost: ${cost:.2f}")

Total tokens: 18254 - cost: $0.01


In [ ]:
# Import chroma
from langchain.vectorstores import Chroma

# Import OpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings

# Create the mebedding function
embedding_function = OpenAIEmbeddings(openai_api_key= api_key)

# Create a database from the documents and embedding function
db = Chroma.from_documents(documents=documents, embedding=embedding_function, persist_directory="my-embeddings")

# Persist the data to disk
db.persist()

In [ ]:
# Import
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI

# Set the question variable
question = "¿Cual es el handle de twitter del consulado de italia en mendoza?"
#question = "¿Quando sono i proximi appuntamenti per cittadinanza del consolato d' italia en mendoza?"
#question = "Que pasos debo realizar  para pedir la ciudadania por matrimonio? ¿Puedes ponerlo en una lista en viñetas?"
#question = "Que requisitos debo cumplir  para pedir la ciudadania por matrimonio? ¿Puedes ponerlo en una lista en viñetas?"
#question = "¿cuanto demora el certificado de antecedentes penales?"

# Query the database as store the results as `context_docs`
context_docs = db.similarity_search(question)

# Create a prompt with 2 variables: `context` and `question`
prompt = PromptTemplate(
    template=""""Usa i seguenti elementi di contesto per rispondere alla domanda alla fine. Se non conosci la risposta, dì semplicemente che non lo sai, non cercare di inventare una risposta.

<context>
{context}
</context>

Domanda: {question}
Risposta:""",
    input_variables=["context", "question"]
)

prompt = PromptTemplate(
    template=""""Usa los siguientes elementos de contexto para responder a la pregunta al final. Si no conoces la respuesta, di que no lo sabes,no inventes una respuesta.

<context>
{context}
</context>

Domanda: {question}
Respuesta:""",
    input_variables=["context", "question"]
)

# Create an LLM with ChatOpenAI
llm = ChatOpenAI(openai_api_key=api_key,temperature=0)

# Create the chain
qa_chain = LLMChain(llm=llm, prompt=prompt)

# Call the chain
result = qa_chain({
    "question": question,
    "context": "\n".join([doc.page_content for doc in context_docs])
})

# Print the result
print(result["text"])